In [1]:
import pandas as pd
import glob

csv_dir_path = "../WebScraper/seedly-forum/*.csv"

In [2]:
def read_csv(filename,user_list):
    
    file_df = pd.read_csv(filename)
    return get_users(file_df,user_list)
    

In [3]:
def check_name(user_list, name):
    for i,user_dict in enumerate(user_list):
        if name in user_dict.keys():
            return i,user_dict
           
    return 0,None

In [4]:
def clean_name(name):
    if "DeleteEdit" in name:
        name = name.replace("DeleteEdit","")
    if "Top Contributor (Sep)" in name:
        name = name.replace("Top Contributor (Sep)","")
    if name.endswith("L"):
        name = name[:-1]
    if "Jonathan Chia Guangrong" in name:
        name = name[:name.find("Jonathan Chia Guangrong")]
    return name.strip()

In [ ]:
def get_users(df,user_list):
    
    for i,content in enumerate(df.content):
        user_dict = {}
        if i == 0:
            substr_index = content.find("Asked by")
            user_details = content[substr_index:]
            new_sub_index = user_details.find("Asked on")
            question_name = user_details[:new_sub_index].replace("Asked by","").strip()
            if question_name.find("Asked")!= -1:
                question_name = question_name[:question_name.find("Asked")]      
        elif i > 1:
            substr_index = content.find("Answered")
            user_details = content[:substr_index]
            commar_index = user_details.find(",")
            level_index = user_details.find("Level")
            rank_index = user_details.find(".")
            name = user_details[:commar_index]
            level_index_2 = name.find("Level")
            if level_index_2 != -1:
                name = name[:level_index_2+1]
            level = user_details[level_index:rank_index].replace("Level","").strip()
            name = clean_name(name)
            index, user_dict = check_name(user_list,name)
            if user_dict:
                user_list.pop(index)
                user_dict[name]["counter"] += 1
                user_list.append(user_dict)
            else:
                user_dict = {name:{"level":level,"counter":0}}
                user_list.append(user_dict)
            
    return question_name, user_list

In [ ]:
user_list = []
question_user_list = []

for filename in glob.glob(csv_dir_path):
    question_asked, user_list = read_csv(filename,user_list)
    question_user_list.append(question_asked)

In [128]:
def split_user_details(user_details):
    user_details = user_details.replace("Answered on","")
    user_details = user_details.replace("Updated on","")
    commar_index = user_details.find(",")
    level_index = user_details.find("Level")
    level_rank = 0
    if level_index != -1:
        level_rank = user_details[level_index:]
    if commar_index != -1:
        username = user_details[:commar_index+1]
    else:
        username = user_details[:level_index+1]
        
    username = username.replace("Top Contributor (Sep)","")
    comments_index = username.find("comments")
    if comments_index != -1:
        username = username[comments_index+9:]
    if username.endswith("L"):
        username = username[:-1]
    username = username.replace(",","")
    
    if level_rank!=0:
        dot_substring = level_rank.find(".")
        level_substring = level_rank.find("Level")
        level = int(level_rank[level_substring+6:dot_substring])
        rank = level_rank[dot_substring+2:]
    else:
        level = 0
        rank = "Nub"
    
    return username,level,rank
        

In [137]:
import re

question_list = []
answer_list = []
check_answer_list = []
regex = r'\d{2}\s(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s\d{4}'

for filename in glob.glob(csv_dir_path):
    file_df = pd.read_csv(filename)
    for i,content in enumerate(file_df.content):
        question_dict = {}
        answer_dict = {}
        if i == 0:
            substr_index = content.find("Asked by")
            q_user_details = content[substr_index:]
            new_sub_index = q_user_details.find("Asked on")
            question_name = q_user_details[:new_sub_index].replace("Asked by","").strip()
            if question_name.find("Asked")!= -1:
                question_name = question_name[:question_name.find("Asked")]
            question = user_details[new_sub_index+20:]
            question = question[:question.find("answer")-2]
            question_dict[question_name] = question
        elif i > 1:
            #Extract Answer and user details
            substr_index = content.find("Answered")
            pos_substr_index = content.find("Updated")
            date = re.findall(regex,content)
            if date:
                date_substr = content.find(date[0])
                user_details = content[:date_substr]
                answer = content[date_substr+11:]
                answer = answer[:answer.find("comments")-2]
            elif substr_index != -1:
                answer = content[substr_index:]
                answer = answer[:answer.find("comments")-2]
                w_substr = answer.find("w ago")
                h_substr = answer.find("h ago")
                d_substr = answer.find("d ago")
                if w_substr!= -1:
                    answer = answer[w_substr+5:]
                elif h_substr!= -1:
                    answer = answer[h_substr+5:]
                elif d_substr != -1:
                    answer = answer[d_substr+5:]
                user_details = content[:substr_index]
            elif pos_substr_index!= -1:
                answer = content[pos_substr_index:]
                answer = answer[:answer.find("comments")-2]
                w_substr = answer.find("w ago")
                h_substr = answer.find("h ago")
                d_substr = answer.find("d ago")
                if w_substr!= -1:
                    answer = answer[w_substr+5:]
                elif h_substr!= -1:
                    answer = answer[h_substr+5:]
                elif d_substr != -1:
                    answer = answer[d_substr+5:]
                user_details = content[:pos_substr_index]
            else:
                w_substr = content.find("w ago")
                h_substr = content.find("h ago")
                d_substr = content.find("d ago")
                if w_substr!= -1:
                    answer = content[w_substr+5:]
                    user_details = content[:w_substr-1]
                elif h_substr!= -1:
                    answer = content[h_substr+5:]
                    user_details = content[:h_substr-1]
                elif d_substr != -1:
                    answer = content[d_substr+5:]
                    user_details = content[:d_substr-1]
        
        username, level, rank = split_user_details(user_details)
        
        if answer not in check_answer_list:
            check_answer_list.append(answer)
            answer_dict["username"] = username
            answer_dict["level"] = level
            answer_dict["rank"] = rank
            answer_dict["answer"] = answer
            answer_list.append(answer_dict)
        
        question_list.append(question_dict)
        

        

In [138]:


answer_df = pd.DataFrame(answer_list)

In [139]:
answer_df

,username,level,rank,answer
0,Lava Ama A Dios,2,Rookie,So let’s weigh the plans of zero and zero1 ass...
1,Jason Yip,2,Rookie,All condos have en-bloc potential if you have ...
2,Billy Ko,6,Master,Low Flats / Few UnitsWhen developers redevelop...
3,Douglas Chow,3,Wonderkid,Hi. Douglas here. 2nd keynote speaker.It;s pre...
4,Douglas Chow,3,Wonderkid,"Hi, Douglas here. 2nd Keynote Speaker.I'm afra..."
...,...,...,...,...
17486,RyaN Cheong,2,Rookie,You may refer to following:https://www.google....
17487,,0,Nub,My referral code for revolut:\r\nhttps://revol...
17488,Leslie Koh,4,Prodigy,"If you’re comparing on features, Revolut is be..."
17489,,0,Nub,"Hi Leslie, may I know how to you do comparison..."
